In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve
random_seed = 10
# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)

CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test = pd.concat(CM_Y_test.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test = pd.concat(CF_Y_test.values(), join='inner', ignore_index=True)


M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]

F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']
    
     
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]


#================================================================================================================================
# parameters space, models definition and random grid search
#================================================================================================================================

# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
scoring = ['f1', 'recall', 'precision']

# XGBoost space
params_xgb = {'learning_rate': np.linspace(0, 1, 11),
              'min_split_loss': np.linspace(0, 1, 6),
              'max_depth': np.linspace(2, 10, 5, dtype=int),
              'min_child_weight': np.linspace(1, 20, 20, dtype=int),
              'colsample_bytree': np.linspace(0.5, 1, 6),
              'reg_alpha': np.linspace(0, 1, 11),
              'scale_pos_weight': np.linspace(4, 50, 24, dtype=int),
              'n_estimators': np.linspace(10, 450, 12, dtype=int),
              'reg_lambda': np.linspace(0, 10, 11),
             }

M_xgb_sample = RandomizedSearchCV(estimator=XGBClassifier(booster='gbtree',
                                                                     objective='binary:logistic',
                                                                     random_state=random_seed,
                                                                     nthread=1,
                                                                     verbosity=1),
                                             param_distributions=params_xgb,
                                             n_iter=50,
                                             scoring=scoring,
                                             cv=kfold,
                                             n_jobs=-1,
                                             random_state=random_seed,
                                             verbose=2,
                                             refit='f1')
F_xgb_sample = RandomizedSearchCV(estimator=XGBClassifier(booster='gbtree',
                                                                     objective='binary:logistic',
                                                                     random_state=random_seed,
                                                                     nthread=1,
                                                                     verbosity=1),
                                             param_distributions=params_xgb,
                                             n_iter=50,
                                             scoring=scoring,
                                             cv=kfold,
                                             n_jobs=-1,
                                             random_state=random_seed,
                                             verbose=2,
                                             refit='f1')

M_xgb_sample.fit(M_X_train_ready_selected, M_Y_train_ready)
dump(M_xgb_sample, "xgb_sample\M_xgb_sample_sample_data.joblib")

F_xgb_sample.fit(F_X_train_ready_selected, F_Y_train_ready)
dump(F_xgb_sample, "xgb_sample\F_xgb_sample_sample_data.joblib")


In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


from sklearn.metrics import r2_score

# import matplotlib.pyplot as plt
# %matplotlib inline


random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)


CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test_ready = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test_ready = pd.concat(CM_Y_test_ready.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test_ready = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test_ready = pd.concat(CF_Y_test_ready.values(), join='inner', ignore_index=True)
M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]

F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']
    
     
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]
     
M_xgb_selected = load("xgb_sample\M_xgb_sample_sample_data.joblib")
F_xgb_selected = load("xgb_sample\F_xgb_sample_sample_data.joblib")
CM_xgb_selected = load("xgb_sample\M_xgb_sample_sample_data.joblib")
CF_xgb_selected = load("xgb_sample\F_xgb_sample_sample_data.joblib")

M_best_index = M_xgb_selected.best_index_
F_best_index = F_xgb_selected.best_index_

M_Y_train_selected_pred = M_xgb_selected.predict(M_X_train_ready_selected)
# M_Y_train_selected_pred_thresh = (M_xgb_selected.predict_proba(M_X_train_selected)[:,1] >= 0.3).astype(bool) # set threshold as 0.3
F_Y_train_selected_pred = F_xgb_selected.predict(F_X_train_ready_selected)

print('Confusion matrix on the  M train dataset:', confusion_matrix(M_Y_train_ready, M_Y_train_selected_pred))
print('Average F1 score on the  M train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_f1'][M_best_index]))
print('CI F1 score on the M train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_f1'][M_best_index]))
print('Average Sensitivity on the M train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_recall'][M_best_index]))
print('CI Sensitivity on the M train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_recall'][M_best_index]))
print('Average Positive Predictive Value on the M train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_precision'][M_best_index]))
print('CI Positive Predictive Value on the M train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_precision'][M_best_index]))
print('Parameters', M_xgb_selected.best_estimator_)


print('Confusion matrix on the F train dataset:', confusion_matrix(F_Y_train_ready, F_Y_train_selected_pred))
print('Average F1 score on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_f1'][F_best_index]))
print('CI F1 score on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_f1'][F_best_index]))
print('Average Sensitivity on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_recall'][F_best_index]))
print('CI Sensitivity on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_recall'][F_best_index]))
print('Average Positive Predictive Value on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_precision'][F_best_index]))
print('CI Positive Predictive Value on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_precision'][F_best_index]))
print('Parameters', F_xgb_selected.best_estimator_)

from matplotlib import pyplot   
M_Y_test_selected_pred = M_xgb_selected.predict(M_X_test_ready_selected)
F_Y_test_selected_pred = F_xgb_selected.predict(F_X_test_ready_selected)
CM_Y_test_selected_pred = M_xgb_selected.predict(CM_test_ready_selected)
CF_Y_test_selected_pred = F_xgb_selected.predict(CF_test_ready_selected)



print('Confusion matrix on the M test dataset:', confusion_matrix(M_Y_test_ready, M_Y_test_selected_pred))
print('F1 score on the M test dataset:', "{:.1%}".format(f1_score(M_Y_test_ready, M_Y_test_selected_pred)))

print('Sensitivity on the M test dataset:', "{:.1%}".format(recall_score(M_Y_test_ready, M_Y_test_selected_pred)))
print('Positive Predictive Value on the M test dataset:', "{:.1%}".format(precision_score(M_Y_test_ready, M_Y_test_selected_pred)))


print('Confusion matrix on the F test dataset:', confusion_matrix(F_Y_test_ready, F_Y_test_selected_pred))
print('F1 score on the F test dataset:', "{:.1%}".format(f1_score(F_Y_test_ready, F_Y_test_selected_pred)))
print('Sensitivity on the F test dataset:', "{:.1%}".format(recall_score(F_Y_test_ready, F_Y_test_selected_pred)))
print('Positive Predictive Value on the F test dataset:', "{:.1%}".format(precision_score(F_Y_test_ready, F_Y_test_selected_pred)))

print('Confusion matrix on the CF  test dataset:', confusion_matrix(CF_Y_test_ready, CF_Y_test_selected_pred))
print('F1 score on the CF test dataset:', "{:.1%}".format(f1_score(CF_Y_test_ready, CF_Y_test_selected_pred)))
print('Sensitivity on the CF test dataset:', "{:.1%}".format(recall_score(CF_Y_test_ready, CF_Y_test_selected_pred)))
print('Positive Predictive Value on the CF test dataset:', "{:.1%}".format(precision_score(CF_Y_test_ready, CF_Y_test_selected_pred)))

print('Confusion matrix on the CM test dataset:', confusion_matrix(CM_Y_test_ready, CM_Y_test_selected_pred))
print('F1 score on the CM test dataset:', "{:.1%}".format(f1_score(CM_Y_test_ready, CM_Y_test_selected_pred)))
print('Sensitivity on the CM test dataset:', "{:.1%}".format(recall_score(CM_Y_test_ready, CM_Y_test_selected_pred)))
print('Positive Predictive Value on the CM test dataset:', "{:.1%}".format(precision_score(CM_Y_test_ready, CM_Y_test_selected_pred)))



def pr_f1_curve(model, dx_test, dy_test):
         dy_test_predict_proba = model.predict_proba(dx_test)[:,1]
         dy_test_predict = model.predict(dx_test)
         ppv_curve, sensitivity_curve, _ = precision_recall_curve(dy_test, dy_test_predict_proba)
         F1_score, AUC_score = f1_score(dy_test, dy_test_predict), auc(sensitivity_curve, ppv_curve)
         print(': F1 score = %.2f - Area under Curve = %.2f' % (F1_score, AUC_score))
         prevalence = np.sum(dy_test)/len(dy_test)
         pyplot.plot([0, 1], [prevalence, prevalence], linestyle='--', label='Random Testing')
         pyplot.plot(sensitivity_curve, ppv_curve, marker='.', label='Algorithmic Testing')
         pyplot.plot(sensitivity_curve, 2*(sensitivity_curve*ppv_curve)/(sensitivity_curve+ppv_curve), marker='', label='F1')
         pyplot.xlabel('Sensitivity')
         pyplot.ylabel('Positive Predicted Value')
         pyplot.legend(bbox_to_anchor=(0.45, 0.75))
         pyplot.title('Precisio Recall Curve-Males')   
         pyplot.show()
        
    
        
pr_f1_curve(M_xgb_selected, M_X_test_ready_selected, M_Y_test_ready)
# pr_f1_curve(F_xgb_selected, F_X_test_ready_selected, F_Y_test_ready)
# pr_f1_curve(CF_xgb_selected, CF_test_ready_selected, CF_Y_test_ready)
# pr_f1_curve(CM_xgb_selected, CM_test_ready_selected, CM_Y_test_ready)


# Explain model predictions using shapley value
# M_explainer = shap.TreeExplainer(M_xgb_selected.best_estimator_)
# M_shap_values = M_explainer.shap_values(M_X_train_ready_selected)
# F_explainer = shap.TreeExplainer(F_xgb_selected.best_estimator_)
# F_shap_values = F_explainer.shap_values(F_X_train_ready_selected)

# CF_explainer = shap.TreeExplainer(CF_xgb_selected.best_estimator_)
# CF_shap_values = CF_explainer.shap_values(CF_test_ready_selected)
# 
# CM_explainer = shap.TreeExplainer(CM_xgb_selected.best_estimator_)
# CM_shap_values = CM_explainer.shap_values(CM_test_ready_selected)

# Plot summary_plot
# shap.summary_plot(M_shap_values, M_X_train_ready_selected, max_display=24)
# shap.summary_plot(F_shap_values, F_X_train_ready_selected, max_display=29)

# shap.summary_plot(CM_shap_values, CM_test_ready_selected, max_display=24)
# shap.summary_plot(CF_shap_values, CF_test_ready_selected, max_display=29)

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


from sklearn.metrics import r2_score

# import matplotlib.pyplot as plt
# %matplotlib inline


random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)


CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test_ready = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test_ready = pd.concat(CM_Y_test_ready.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test_ready = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test_ready = pd.concat(CF_Y_test_ready.values(), join='inner', ignore_index=True)
M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]

F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']
    
     
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]
     
M_xgb_selected = load("xgb_sample\M_xgb_sample_sample_data.joblib")
F_xgb_selected = load("xgb_sample\F_xgb_sample_sample_data.joblib")
CM_xgb_selected = load("xgb_sample\M_xgb_sample_sample_data.joblib")
CF_xgb_selected = load("xgb_sample\F_xgb_sample_sample_data.joblib")

M_best_index = M_xgb_selected.best_index_
F_best_index = F_xgb_selected.best_index_

M_Y_train_selected_pred = M_xgb_selected.predict(M_X_train_ready_selected)
# M_Y_train_selected_pred_thresh = (M_xgb_selected.predict_proba(M_X_train_selected)[:,1] >= 0.3).astype(bool) # set threshold as 0.3
F_Y_train_selected_pred = F_xgb_selected.predict(F_X_train_ready_selected)


from matplotlib import pyplot   
M_Y_test_selected_pred = M_xgb_selected.predict(M_X_test_ready_selected)
F_Y_test_selected_pred = F_xgb_selected.predict(F_X_test_ready_selected)
CM_Y_test_selected_pred = M_xgb_selected.predict(CM_test_ready_selected)
CF_Y_test_selected_pred = F_xgb_selected.predict(CF_test_ready_selected)



def pr_f1_curve(model, dx_test, dy_test):
         dy_test_predict_proba = model.predict_proba(dx_test)[:,1]
         dy_test_predict = model.predict(dx_test)
         ppv_curve, sensitivity_curve, _ = precision_recall_curve(dy_test, dy_test_predict_proba)
         F1_score, AUC_score = f1_score(dy_test, dy_test_predict), auc(sensitivity_curve, ppv_curve)
         print(': F1 score = %.2f - Area under Curve = %.2f' % (F1_score, AUC_score))
         prevalence = np.sum(dy_test)/len(dy_test)
         pyplot.plot([0, 1], [prevalence, prevalence], linestyle='--', label='Random Testing')
         pyplot.plot(sensitivity_curve, ppv_curve, marker='.', label='Algorithmic Testing')
         pyplot.plot(sensitivity_curve, 2*(sensitivity_curve*ppv_curve)/(sensitivity_curve+ppv_curve), marker='', label='F1')
         pyplot.xlabel('Sensitivity')
         pyplot.ylabel('Positive Predicted Value')
         pyplot.legend(bbox_to_anchor=(0.45, 0.75))
         pyplot.title('Precision-Recall Curve-Females')   
         pyplot.show()
            
       
        
    
        
# pr_f1_curve(M_xgb_selected, M_X_test_ready_selected, M_Y_test_ready)
pr_f1_curve(F_xgb_selected, F_X_test_ready_selected, F_Y_test_ready)

In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


from sklearn.metrics import r2_score

# import matplotlib.pyplot as plt
# %matplotlib inline


random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)


CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test_ready = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test_ready = pd.concat(CM_Y_test_ready.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test_ready = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test_ready = pd.concat(CF_Y_test_ready.values(), join='inner', ignore_index=True)
M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]

F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']
    
     
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]
     
M_xgb_selected = load("xgb_sample\M_xgb_sample_sample_data.joblib")
F_xgb_selected = load("xgb_sample\F_xgb_sample_sample_data.joblib")
CM_xgb_selected = load("xgb_sample\M_xgb_sample_sample_data.joblib")
CF_xgb_selected = load("xgb_sample\F_xgb_sample_sample_data.joblib")
M_model = load("xgb_sample\M_xgb_sample_sample_data.joblib")
F_model = load("xgb_sample\F_xgb_sample_sample_data.joblib")

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve


random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)

CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test = pd.concat(CM_Y_test.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test = pd.concat(CF_Y_test.values(), join='inner', ignore_index=True)


M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]

F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']
    
     
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]

# precision-recall curve and f1
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot
# generate 2 class dataset
# X, y = make_classification(n_samples=1000, n_classes=2, random_state=1)
# split into train/test sets
# trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(M_X_train_ready_selected, M_Y_train_ready)
# predict probabilities
lr_probs = model.predict_proba(M_X_test_ready_selected)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# predict class values
yhat = model.predict(M_X_test_ready_selected)
lr_precision, lr_recall, _ = precision_recall_curve(M_Y_test_ready, lr_probs)
lr_f1, lr_auc = f1_score(M_Y_test_ready, yhat), auc(lr_recall, lr_precision)
# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
# plot the precision-recall curves
no_skill = len(M_Y_test_ready[M_Y_test_ready==1]) / len(M_Y_test_ready)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(lr_recall, lr_precision, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
import sklearn.metrics as sk_metrics
    
def f_beta_by_threshold(y_true, y_pred_pos, beta, thres_nr=100):
    thresholds = [i / thres_nr for i in range(1, thres_nr, 1)]

    f_scores = []
    for thres in thresholds:
        y_pred_class = y_pred_pos > thres
        score = sk_metrics.fbeta_score(y_true, y_pred_class, beta=beta)
        f_scores.append(score)

    return thresholds, f_scores

betas = [0.1, 0.25, 0.5, 1.0, 1.25, 1.5, 2, 5, 10]

fig, ax = pyplot.subplots()
for beta in betas:
    thresholds, scores = f_beta_by_threshold(M_Y_test_ready, M_Y_test_ready_pred[:, 1], beta)
    ax.plot(thresholds, scores, label='beta={}'.format(beta))
    ax.set_title('F beta(threshold) by beta')
    ax.set_xlabel('threshold')
    ax.set_ylabel('F beta')
    ax.legend(loc='right', bbox_to_anchor=(1.3,0.7))
pyplot.show()

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)
M_model = load("xgb_sample\M_xgb_sample_sample_data.joblib")
F_model = load("xgb_sample\F_xgb_sample_sample_data.joblib")
M_precision, M_recall, M_thresholds = precision_recall_curve(M_Y_test_ready, M_model.predict_proba(M_X_test_ready_selected)[:,1])

M_f1 = 2*M_recall*M_precision/(M_recall+M_precision)

F_precision, F_recall, F_thresholds = precision_recall_curve(F_Y_test_ready, F_model.predict_proba(F_X_test_ready_selected)[:,1])

F_f1 = 2*F_recall*F_precision/(F_recall+F_precision)


M_index = np.argmax(M_precision[(M_recall>=0.949999999) & (M_recall >= 0.9500000001)])

print('Males')
print('Best f1: ', M_f1[M_index])
print('Best precision: ', M_precision[M_index])
print('Best threshold: ', M_thresholds[M_index])

F_index = np.argmax(F_precision[(F_recall>=0.949999999) & (F_recall >= 0.9500000001)])

print('Females')
print('Best f1: ', F_f1[F_index])
print('Best precision: ', F_precision[F_index])
print('Best threshold: ', F_thresholds[F_index])

In [ ]:
from sklearn.metrics import confusion_matrix
M_Y_test_pred_thresh = (M_model.predict_proba(M_X_test_ready_selected)[:,1] >= 0.00661738).astype(bool)

print('Males')
confusion_matrix(M_Y_test_ready, M_Y_test_pred_thresh)

F_Y_test_pred_thresh = (F_model.predict_proba(F_X_test_ready_selected)[:,1] >= 0.05414463).astype(bool)

print('Females')
confusion_matrix(F_Y_test_ready, F_Y_test_pred_thresh)

In [ ]:
M_Y_test_pred_thresh = (M_model.predict_proba(M_X_test_ready_selected)[:,1] >= 0.95).astype(bool)

print('Males')
confusion_matrix(M_Y_test_ready, M_Y_test_pred_thresh)


F_Y_test_pred_thresh = (F_model.predict_proba(F_X_test_ready_selected)[:,1] >= 0.95).astype(bool)

print('Females')
confusion_matrix(F_Y_test_ready, F_Y_test_pred_thresh)

In [ ]:
print('Best F1-Score: ', np.max(M_f1))
print('Best precision: ', M_precision[np.argmax(M_f1)])
print('Best recall: ', M_recall[np.argmax(M_f1)])
print('Best threshold: ', M_thresholds[np.argmax(M_f1)])

print('Best F1-Score: ', np.max(F_f1))
print('Best precision: ', F_precision[np.argmax(F_f1)])
print('Best recall: ', F_recall[np.argmax(F_f1)])
print('Best threshold: ', F_thresholds[np.argmax(F_f1)])

In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)
# code for uploading pickled data

# code for uploading pickled data
# M_X_test_entFe = pickle.load(open("pure_data\M_X_test_entFe.pkl", 'rb'))
# M_Y_test_entFe = pickle.load(open("pure_data\M_Y_test_entFe.pkl", 'rb'))
M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

# F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
# F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
# F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
# F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)


# stratified k-fold and model initialization
# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

M_model = load("xgb_sample\M_xgb_sample_sample_data.joblib")
# F_model = load("xgb_sample\F_xgb_sample_sample_data.joblib")

M_sfs_samples = SFS(M_model.best_estimator_, k_features=27, forward=True, floating=False, scoring='f1', verbose=1, cv=0, n_jobs=-1)
# F_sfs = SFS(F_model.best_estimator_, k_features=35, forward=True, floating=True, scoring='f1', verbose=1, cv=0, n_jobs=-1)

M_sfs_samples.fit(M_X_train_ready, M_Y_train_ready)
# F_sfs.fit(F_X_train_ready, F_Y_train_ready)

dump(M_sfs_samples, 'M_sfs_samples.joblib')
# dump(F_sfs_samples, 'F_sfs_samples.joblib')

In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)
# code for uploading pickled data

# code for uploading pickled data
# M_X_test_entFe = pickle.load(open("pure_data\M_X_test_entFe.pkl", 'rb'))
# M_Y_test_entFe = pickle.load(open("pure_data\M_Y_test_entFe.pkl", 'rb'))
# M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
# M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
# M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
# M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)


# stratified k-fold and model initialization
# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

# M_model = load("xgb_sample\M_xgb_sample_sample_data.joblib")
F_model = load("xgb_sample\F_xgb_sample_sample_data.joblib")

# M_sfs = SFS(M_model.best_estimator_, k_features=27, forward=True, floating=False, scoring='f1', verbose=1, cv=0, n_jobs=-1)
F_sfs_samples = SFS(F_model.best_estimator_, k_features=35, forward=True, floating=True, scoring='f1', verbose=1, cv=0, n_jobs=-1)

# M_sfs.fit(M_X_train_ready, M_Y_train_ready)
F_sfs_samples.fit(F_X_train_ready, F_Y_train_ready)

# dump(M_sfs_samples, 'M_sfs_samples.joblib')
dump(F_sfs_samples, 'F_sfs_samples.joblib')

In [ ]:

import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve


random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)
M_sfs_samples.subsets_
# F_sfs_samples.subsets_

In [ ]:
M_sfs_samples = load("M_sfs_samples.joblib")
M_sfs_samples.subsets_
# F_sfs_samples = load("F_sfs_samples.joblib")
# F_sfs_samples.subsets_

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
fig1 = plot_sfs(M_sfs_samples.get_metric_dict(),
                kind='std_dev',
                figsize=(22, 11))

plt.ylim([0.3, 1])
plt.ylabel('f1 score')
plt.title('Sequential Forward Selection')
plt.grid()
plt.show

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
fig1 = plot_sfs(F_sfs_samples.get_metric_dict(),
                kind='std_dev',
                figsize=(22, 11))

plt.ylim([0.85, 1])
plt.ylabel('f1 score')
plt.title('Sequential Forward Selection')
plt.grid()
plt.show

In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

plot_sfs(M_sfs_samples.get_metric_dict(),
                kind='std_dev',
                figsize=(10, 6))

plt.ylim([0.3, 1])
plt.xlabel('Number of variables', fontsize=20)
plt.ylabel('f1 score', fontsize=20)
plt.title('SFFS - males', fontsize=20)
plt.grid()
plt.savefig('M_sfs_samples.jpg')


In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import matplotlib.pyplot as plt
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data
M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)


# select features
M_selected_features = ['Age',
   'Relationship with family head',
   'Respondent live in household',
   'Sick to work last three months',
   'Highest level of education',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'CFcumcision status',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Alcohol drink frequency',
   'Urban area indicator',
   'Known HIV status',
   'Wealth quintile',
   'Sexual intercourse in the past 12 months',
   'Bought/sold sex in the past 12 months',
   'Condom was used at last paid sex in past 12 months',
   'last sex partner relations past 12 months',
   'LAg: recent/long term infection',
   'CD4 category',
   'ART initiated past 12 months those ever on ART']  
M_X_train_selected = M_X_train_ready[M_selected_features]

F_selected_features = ['Age',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'Ever married/lived together',
   'Number of pregnancies',
   'No. of childen given bFth since 2012',
   'Avoiding pregnancy',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Urban area indicator',
   'Known HIV status',
   'Delivered 12 months preceeding survey',
   'Self-reported ARV status during pregnancy',
   'Sexual intercourse past 12 months',
   'Ever had sexual intercourse',
   'Condom was used at last paid sex in past 12 months',
   'LAg: recent/long term infection',
   'On ART',
   'ART initiated past 12 months those ever on ART',
   'Whether ARVs detected']    
     
F_X_train_selected = F_X_train_ready[F_selected_features]

#================================================================================================================================
# parameters space, models definition and random grid search
#================================================================================================================================

# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
scoring = ['f1', 'recall', 'precision']

# XGBoost space
params_xgb = {'learning_rate': np.linspace(0, 1, 11),
              'min_split_loss': np.linspace(0, 1, 6),
              'max_depth': np.linspace(2, 10, 5, dtype=int),
              'min_child_weight': np.linspace(1, 20, 20, dtype=int),
              'colsample_bytree': np.linspace(0.5, 1, 6),
              'reg_alpha': np.linspace(0, 1, 11),
              'scale_pos_weight': np.linspace(4, 50, 24, dtype=int),
              'n_estimators': np.linspace(10, 450, 12, dtype=int),
              'reg_lambda': np.linspace(0, 10, 11),
             }

M_xgb_selected_21_samples = RandomizedSearchCV(estimator=XGBClassifier(booster='gbtree',
                                                                     objective='binary:logistic',
                                                                     random_state=random_seed,
                                                                     nthread=1,
                                                                     verbosity=1),
                                             param_distributions=params_xgb,
                                             n_iter=250,
                                             scoring=scoring,
                                             cv=kfold,
                                             n_jobs=-1,
                                             random_state=random_seed,
                                             verbose=2,
                                             refit='f1')

F_xgb_selected_20 = RandomizedSearchCV(estimator=XGBClassifier(booster='gbtree',
                                                                     objective='binary:logistic',
                                                                     random_state=random_seed,
                                                                     nthread=1,
                                                                     verbosity=1),
                                             param_distributions=params_xgb,
                                             n_iter=250,
                                             scoring=scoring,
                                             cv=kfold,
                                             n_jobs=-1,
                                             random_state=random_seed,
                                             verbose=2,
                                             refit='f1')

M_xgb_selected_21_samples.fit(M_X_train_selected, M_Y_train_ready)
dump(M_xgb_selected_21_samples, "xgb_selected\M_xgb_selected_21_sample_data.joblib")

F_xgb_selected_20.fit(F_X_train_selected, F_Y_train_ready)
dump(F_xgb_selected_20, "xgb_selected\F_xgb_selected_20_sample_data.joblib")

In [3]:
%matplotlib inline

# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.metrics import precision_recall_curve
import bisect 
from scipy.stats import mstats
import matplotlib.ticker as plticker

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data
M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)

# select features
# select features
M_selected_features = ['Age',
   'Relationship with family head',
   'Respondent live in household',
   'Sick to work last three months',
   'Highest level of education',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'CFcumcision status',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Alcohol drink frequency',
   'Urban area indicator',
   'Known HIV status',
   'Wealth quintile',
   'Sexual intercourse in the past 12 months',
   'Bought/sold sex in the past 12 months',
   'Condom was used at last paid sex in past 12 months',
   'last sex partner relations past 12 months',
   'LAg: recent/long term infection',
   'CD4 category',
   'ART initiated past 12 months those ever on ART']  
M_X_train_selected = M_X_train_ready[M_selected_features]

F_selected_features = ['Age',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'Ever married/lived together',
   'Number of pregnancies',
   'No. of childen given bFth since 2012',
   'Avoiding pregnancy',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Urban area indicator',
   'Known HIV status',
   'Delivered 12 months preceeding survey',
   'Self-reported ARV status during pregnancy',
   'Sexual intercourse past 12 months',
   'Ever had sexual intercourse',
   'Condom was used at last paid sex in past 12 months',
   'LAg: recent/long term infection',
   'On ART',
   'ART initiated past 12 months those ever on ART',
   'Whether ARVs detected']    
     
F_X_train_selected = F_X_train_ready[F_selected_features]

M_model_21 = load("xgb_selected\M_xgb_selected_21_sample_data.joblib")
F_model_20 = load("xgb_selected\F_xgb_selected_20_sample_data.joblib")

In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from matplotlib import pyplot
%matplotlib inline
import matplotlib.pyplot as plt
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data
M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)
# select features
# select features
M_selected_features = ['Age',
   'Relationship with family head',
   'Respondent live in household',
   'Sick to work last three months',
   'Highest level of education',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'CFcumcision status',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Alcohol drink frequency',
   'Urban area indicator',
   'Known HIV status',
   'Wealth quintile',
   'Sexual intercourse in the past 12 months',
   'Bought/sold sex in the past 12 months',
   'Condom was used at last paid sex in past 12 months',
   'last sex partner relations past 12 months',
   'LAg: recent/long term infection',
   'CD4 category',
   'ART initiated past 12 months those ever on ART']  
M_X_train_selected = M_X_train_ready[M_selected_features]

F_selected_features = ['Age',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'Ever married/lived together',
   'Number of pregnancies',
   'No. of childen given bFth since 2012',
   'Avoiding pregnancy',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Urban area indicator',
   'Known HIV status',
   'Delivered 12 months preceeding survey',
   'Self-reported ARV status during pregnancy',
   'Sexual intercourse past 12 months',
   'Ever had sexual intercourse',
   'Condom was used at last paid sex in past 12 months',
   'LAg: recent/long term infection',
   'On ART',
   'ART initiated past 12 months those ever on ART',
   'Whether ARVs detected']    
     
F_X_train_selected = F_X_train_ready[F_selected_features]
M_X_test_selected = M_X_test_ready[M_selected_features]
F_X_test_selected = F_X_test_ready[F_selected_features]
     
M_xgb_selected = load("xgb_selected\M_xgb_selected_21_sample_data.joblib")
F_xgb_selected = load("xgb_selected\F_xgb_selected_20_sample_data.joblib")

M_best_index = M_xgb_selected.best_index_
F_best_index = F_xgb_selected.best_index_

M_Y_train_selected_pred = M_xgb_selected.predict(M_X_train_selected)
# M_Y_train_selected_pred_thresh = (M_xgb_selected.predict_proba(M_X_train_selected)[:,1] >= 0.3).astype(bool) # set threshold as 0.3
F_Y_train_selected_pred = F_xgb_selected.predict(F_X_train_selected)

print('Confusion matrix on the train dataset:', confusion_matrix(M_Y_train_ready, M_Y_train_selected_pred))
print('Average F1 score on the train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_f1'][M_best_index]))
print('CI F1 score on the train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_f1'][M_best_index]))
print('Average Sensitivity on the train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_recall'][M_best_index]))
print('CI Sensitivity on the train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_recall'][M_best_index]))
print('Average Positive Predictive Value on the train sample', "{:.1%}".format(M_xgb_selected.cv_results_['mean_test_precision'][M_best_index]))
print('CI Positive Predictive Value on the train sample', "{:.1%}".format(2*M_xgb_selected.cv_results_['std_test_precision'][M_best_index]))
print('Parameters', M_xgb_selected.best_estimator_)

print('Confusion matrix on the F train dataset:', confusion_matrix(F_Y_train_ready, F_Y_train_selected_pred))
print('Average F1 score on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_f1'][F_best_index]))
print('CI F1 score on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_f1'][F_best_index]))
print('Average Sensitivity on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_recall'][F_best_index]))
print('CI Sensitivity on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_recall'][F_best_index]))
print('Average Positive Predictive Value on the F train sample', "{:.1%}".format(F_xgb_selected.cv_results_['mean_test_precision'][F_best_index]))
print('CI Positive Predictive Value on the F train sample', "{:.1%}".format(2*F_xgb_selected.cv_results_['std_test_precision'][F_best_index]))
print('Parameters', F_xgb_selected.best_estimator_)
     
M_Y_test_selected_pred = M_xgb_selected.predict(M_X_test_selected)
F_Y_test_selected_pred = F_xgb_selected.predict(F_X_test_selected)

print('Confusion matrix on the test dataset:', confusion_matrix(M_Y_test_ready, M_Y_test_selected_pred))
print('F1 score on the test dataset:', "{:.1%}".format(f1_score(M_Y_test_ready, M_Y_test_selected_pred)))
print('Sensitivity on the test dataset:', "{:.1%}".format(recall_score(M_Y_test_ready, M_Y_test_selected_pred)))
print('Positive Predictive Value on the test dataset:', "{:.1%}".format(precision_score(M_Y_test_ready, M_Y_test_selected_pred)))

print('Confusion matrix on the F test dataset:', confusion_matrix(F_Y_test_ready, F_Y_test_selected_pred))
print('F1 score on the F test dataset:', "{:.1%}".format(f1_score(F_Y_test_ready, F_Y_test_selected_pred)))
print('Sensitivity on the F test dataset:', "{:.1%}".format(recall_score(F_Y_test_ready, F_Y_test_selected_pred)))
print('Positive Predictive Value on the F test dataset:', "{:.1%}".format(precision_score(F_Y_test_ready, F_Y_test_selected_pred)))


def pr_f1_curve(model, dx_test, dy_test):
     dy_test_predict_proba = model.predict_proba(dx_test)[:,1]
     dy_test_predict = model.predict(dx_test)
     ppv_curve, sensitivity_curve, _ = precision_recall_curve(dy_test, dy_test_predict_proba)
     F1_score, AUC_score = f1_score(dy_test, dy_test_predict), auc(sensitivity_curve, ppv_curve)
     print(': F1 score = %.2f - Area under Curve = %.2f' % (F1_score, AUC_score))
     prevalence = np.sum(dy_test)/len(dy_test)
     pyplot.plot([0, 1], [prevalence, prevalence], linestyle='--', label='Random Testing')
     pyplot.plot(sensitivity_curve, ppv_curve, marker='.', label='Algorithmic Testing')
     pyplot.plot(sensitivity_curve, 2*(sensitivity_curve*ppv_curve)/(sensitivity_curve+ppv_curve), marker='', label='F1')
     pyplot.xlabel('Sensitivity')
     pyplot.ylabel('Positive Predicted Value')
     pyplot.legend(bbox_to_anchor=(0.45, 0.75))
     pyplot.show()

pr_f1_curve(M_xgb_selected, M_X_test_selected, M_Y_test_ready)
pr_f1_curve(F_xgb_selected, F_X_test_selected, F_Y_test_ready)

# Explain model predictions using shapley value
M_explainer = shap.TreeExplainer(M_xgb_selected.best_estimator_)
M_shap_values = M_explainer.shap_values(M_X_train_selected)
F_explainer = shap.TreeExplainer(F_xgb_selected.best_estimator_)
F_shap_values = F_explainer.shap_values(F_X_train_selected)

# Plot summary_plot
M_shap = shap.summary_plot(M_shap_values, M_X_train_selected, max_display=21)
F_shap = shap.summary_plot(F_shap_values, F_X_train_selected, max_display=20)
plt.savefig('M_shapsample.jpg')
plt.savefig('F_shapsample.jpg')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
M_shap = shap.summary_plot(M_shap_values, M_X_train_selected, max_display=21)
F_shap = shap.summary_plot(F_shap_values, F_X_train_selected, max_display=20)
plt.savefig('M_shapsample.jpg')
plt.savefig('F_shapsample.jpg')

In [5]:
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from matplotlib import pyplot
%matplotlib inline
import matplotlib.pyplot as plt
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data
M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)
# select features
# select features
M_selected_features = ['Age',
   'Relationship with family head',
   'Respondent live in household',
   'Sick to work last three months',
   'Highest level of education',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'CFcumcision status',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Alcohol drink frequency',
   'Urban area indicator',
   'Known HIV status',
   'Wealth quintile',
   'Sexual intercourse in the past 12 months',
   'Bought/sold sex in the past 12 months',
   'Condom was used at last paid sex in past 12 months',
   'last sex partner relations past 12 months',
   'LAg: recent/long term infection',
   'CD4 category',
   'ART initiated past 12 months those ever on ART']  
M_X_train_selected = M_X_train_ready[M_selected_features]

F_selected_features = ['Age',
   'Highest grade at that school level',
   'Work for payment in last 12 months',
   'Ever married/lived together',
   'Number of pregnancies',
   'No. of childen given bFth since 2012',
   'Avoiding pregnancy',
   'Age at fFst sex',
   'Ever sought TB treatment',
   'Urban area indicator',
   'Known HIV status',
   'Delivered 12 months preceeding survey',
   'Self-reported ARV status during pregnancy',
   'Sexual intercourse past 12 months',
   'Ever had sexual intercourse',
   'Condom was used at last paid sex in past 12 months',
   'LAg: recent/long term infection',
   'On ART',
   'ART initiated past 12 months those ever on ART',
   'Whether ARVs detected']    
     
F_X_train_selected = F_X_train_ready[F_selected_features]
M_X_test_selected = M_X_test_ready[M_selected_features]
F_X_test_selected = F_X_test_ready[F_selected_features]

M_model_21 = load("xgb_selected\M_xgb_selected_21_sample_data.joblib")
F_model_20 = load("xgb_selected\F_xgb_selected_20_sample_data.joblib")
M_precision, M_recall, M_thresholds = precision_recall_curve(M_Y_test_ready, M_model_21.predict_proba(M_X_test_selected)[:,1])

M_f1 = 2*M_recall*M_precision/(M_recall+M_precision)

F_precision, F_recall, F_thresholds = precision_recall_curve(F_Y_test_ready, F_model_20.predict_proba(F_X_test_selected)[:,1])

F_f1 = 2*F_recall*F_precision/(F_recall+F_precision)

In [ ]:
M_index = np.argmax(M_precision[(M_recall>=0.949999999) & (M_recall >= 0.9500000001)])

print('Males')
print('Best f1: ', M_f1[M_index])
print('Best precision: ', M_precision[M_index])
print('Best threshold: ', M_thresholds[M_index])

F_index = np.argmax(F_precision[(F_recall>=0.899999999) & (F_recall >= 0.9000000001)])

print('Females')
print('Best f1: ', F_f1[F_index])
print('Best precision: ', F_precision[F_index])
print('Best threshold: ', F_thresholds[F_index])

In [ ]:
from sklearn.metrics import confusion_matrix
M_Y_test_pred_thresh = (M_model_21.predict_proba(M_X_test_selected)[:,1] >= 0.014001296).astype(bool)

print('Males')
confusion_matrix(M_Y_test_ready, M_Y_test_pred_thresh)

F_Y_test_pred_thresh = (F_model_20.predict_proba(F_X_test_selected)[:,1] >= 0.4615294).astype(bool)

# print('Females')
# confusion_matrix(F_Y_test_ready, F_Y_test_pred_thresh)

In [ ]:
M_Y_test_pred_thresh = (M_model_21.predict_proba(M_X_test_selected)[:,1] >= 0.90).astype(bool)
F_Y_test_pred_thresh = (F_model_20.predict_proba(F_X_test_selected)[:,1] >= 0.90).astype(bool)
print('Males')
confusion_matrix(M_Y_test_ready, M_Y_test_pred_thresh)
# print('Females')
# confusion_matrix(F_Y_test_ready, F_Y_test_pred_thresh)

In [ ]:
print('Best F1-Score: ', np.max(M_f1))
print('Best precision: ', M_precision[np.argmax(M_f1)])
print('Best recall: ', M_recall[np.argmax(M_f1)])
print('Best threshold: ', M_thresholds[np.argmax(M_f1)])

print('Best F1-Score: ', np.max(F_f1))
print('Best precision: ', F_precision[np.argmax(F_f1)])
print('Best recall: ', F_recall[np.argmax(F_f1)])
print('Best threshold: ', F_thresholds[np.argmax(F_f1)])

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


from sklearn.metrics import r2_score

# import matplotlib.pyplot as plt
# %matplotlib inline


random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)

# code for uploading pickled data

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)

CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test_ready = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test_ready = pd.concat(CM_Y_test_ready.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test_ready = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test_ready = pd.concat(CF_Y_test_ready.values(), join='inner', ignore_index=True)
M_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household','Sick to work last three months','Ever attended school','Ever enrolled in school'
                        ,'Highest level of education', 'Highest grade at that school level','Work for payment in last 12 months',
     'Ever married/lived together', 'Avoiding pregnancy','CFcumcision status','Age at fFst sex','Ever sought TB treatment','Alcohol drink frequency',
                        'urban area indicator','Wealth quintile','Sexual intercourse in the past 12 months','Ever had sexual intercourse'] 
 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]

F_selected_features = ['Age', 'Current marital status', 'Number of pregnancies', 'Ever had a pregnancy that resulted in a live bFth',2)
    , 'No. of childen given bFth since 2012', 'Pregnacy status now', 'Known HIV status', 'Relationship with family head','Respondent live in household',
                        'Sick to work last three months','Ever attended school','Ever enrolled in school','Highest level of education', 'Highest grade at that school level',
                        'Work for payment in last 12 months','Ever married/lived together','Avoiding pregnancy',
                        'Age at fFst sex', 'Ever tested HIV','Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile','Sexual intercourse past 12 months','Ever had sexual intercourse']
   
     
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]
     
M_xgb_selected = load("xgb_selected\M_xgb_selected_sample_data.joblib")
F_xgb_selected = load("xgb_sample\F_xgb_sample_sample_data.joblib")
CM_xgb_selected = load("xgb_selected\M_xgb_selected_sample_data.joblib")
CF_xgb_selected = load("xgb_sample\F_xgb_sample_sample_data.joblib")

M_best_index = M_xgb_selected.best_index_
F_best_index = F_xgb_selected.best_index_

M_Y_train_selected_pred = M_xgb_selected.predict(M_X_train_ready_selected)
# M_Y_train_selected_pred_thresh = (M_xgb_selected.predict_proba(M_X_train_selected)[:,1] >= 0.3).astype(bool) # set threshold as 0.3
F_Y_train_selected_pred = F_xgb_selected.predict(F_X_train_ready_selected)




print('Confusion matrix on the M train dataset:', confusion_matrix(M_Y_train_ready, M_Y_train_selected_pred))
print('F1 score on the M train dataset:', "{:.1%}".format(f1_score(M_Y_train_ready, M_Y_train_selected_pred)))
print('Sensitivity on the M train dataset:', "{:.1%}".format(recall_score(M_Y_train_ready, M_Y_train_selected_pred)))
print('Positive Predictive Value on the M train dataset:', "{:.1%}".format(precision_score(M_Y_train_ready, M_Y_train_selected_pred)))


In [ ]:
M_precision, M_recall, M_thresholds = precision_recall_curve(M_Y_test_ready, M_model.predict_proba(M_X_test_selected)[:,1])

M_f1 = 2*M_recall*M_precision/(M_recall+M_precision)

In [ ]:
M_index = np.argmax(M_precision[(M_recall>=0.949999999) & (M_recall >= 0.9500000001)])

print('Males')
print('Best f1: ', M_f1[M_index])
print('Best precision: ', M_precision[M_index])
print('Best threshold: ', M_thresholds[M_index])

In [ ]:
from sklearn.metrics import confusion_matrix
M_Y_test_ready_pred_thresh = (M_model.predict_proba(M_X_test_selected)[:,1] >= 0.0018743522).astype(bool)

print('Males')
confusion_matrix(M_Y_test_ready, M_Y_test_ready_pred_thresh)

In [ ]:
print('Best F1-Score: ', np.max(M_f1))
print('Best precision: ', M_precision[np.argmax(M_f1)])
print('Best recall: ', M_recall[np.argmax(M_f1)])
print('Best threshold: ', M_thresholds[np.argmax(M_f1)])

In [ ]:
M_Y_test_ready_pred_thresh = (M_model.predict_proba(M_X_test_selected)[:,1] >= 0.95).astype(bool)
# F_Y_test_pred_thresh = (F_model.predict_proba(F_X_test)[:,1] >= 0.95).astype(bool)
print('Males')
confusion_matrix(M_Y_test_ready, M_Y_test_ready_pred_thresh)

In [ ]:
def pr_f1_curve(model, dx_test, dy_test):
         dy_test_predict_proba = model.predict_proba(dx_test)[:,1]
         dy_test_predict = model.predict(dx_test)
         ppv_curve, sensitivity_curve, _ = precision_recall_curve(dy_test, dy_test_predict_proba)
         F1_score, AUC_score = f1_score(dy_test, dy_test_predict), auc(sensitivity_curve, ppv_curve)
         print(': F1 score = %.2f - Area under Curve = %.2f' % (F1_score, AUC_score))
         prevalence = np.sum(dy_test)/len(dy_test)
         pyplot.plot([0, 1], [prevalence, prevalence], linestyle='--', label='Random Testing')
         pyplot.plot(sensitivity_curve, ppv_curve, marker='.', label='Algorithmic Testing')
         pyplot.plot(sensitivity_curve, 2*(sensitivity_curve*ppv_curve)/(sensitivity_curve+ppv_curve), marker='', label='F1')
         pyplot.xlabel('Sensitivity')
         pyplot.ylabel('Positive Predicted Value')
         pyplot.legend(bbox_to_anchor=(0.45, 0.75))
         pyplot.show()
        
    
        
pr_f1_curve(M_xgb_selected, M_X_test_ready_selected, M_Y_test_ready)

In [ ]:
M_sfs_pure = load("M_sfs.joblib")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

plot_sfs(M_sfs_pure.get_metric_dict(),
                kind='std_dev',
                figsize=(22, 11))

plt.ylim([0, 1])
plt.xlabel('Number of variables', fontsize=20)
plt.ylabel('f1 score', fontsize=20)
plt.title('Sequential Floating Forward Selection - Males', fontsize=20)
plt.grid()
plt.savefig('M_sfs.jpg')

In [ ]:
# load list of countries
country_list = ['Tanzania', 'Malawi','Zwaziland', 'Zambia']
for name in country_list:
    print('Male')
    print('Country: ' + name)
    print('prevalence train-sample: ' + '{:.1%}'.format(M_Y_train_entFe[name].sum()/len(M_Y_train_entFe[name])))
    print('prevalence test-sample: ' + '{:.1%}'.format(M_Y_test_entFe[name].sum()/len(M_Y_test_entFe[name])))
    print('prevalence validation-sample: ' + '{:.1%}'.format(CM_Y_test[name].sum()/len(CM_Y_test[name])))


for name in country_list:
    print('Female')
    print('Country: ' + name)
    print('prevalence train-sample: ' + '{:.1%}'.format(F_Y_train_entFe[name].sum()/len(F_Y_train_entFe[name])))
    print('prevalence test-sample: ' + '{:.1%}'.format(F_Y_test_entFe[name].sum()/len(F_Y_test_entFe[name])))
    print('prevalence validation-sample: ' + '{:.1%}'.format(CF_Y_test_entFe[name].sum()/len(CF_Y_test_entFe[name])))




In [ ]:
# import libraries
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load
#from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# specifying constant parameters
random_seed = 10

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)
# code for uploading pickled data

# code for uploading pickled data
# M_X_test_entFe = pickle.load(open("pure_data\M_X_test_entFe.pkl", 'rb'))
# M_Y_test_entFe = pickle.load(open("pure_data\M_Y_test_entFe.pkl", 'rb'))
M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

M_model = load("xgb_selected\M_xgb_selected_sample_data.joblib")
# F_model = load("xgb_entFe_pure_data/M_xgb_entFe_pure_data.joblib")

M_sfs_sample = SFS(M_model.best_estimator_, k_features=20, forward=True, floating=False, scoring='f1', verbose=1, cv=kfold, n_jobs=-1)
# F_sfs = SFS(F_model.best_estimator_, k_features=30, forward=True, floating=True, scoring='f1', verbose=1, cv=0, n_jobs=-1)

M_sfs_sample.fit(M_X_train_ready, M_Y_train_ready)
# F_sfs.fit(F_X_train_entFe, F_Y_train_entFe)

dump(M_sfs_sample, 'M_sfs_sample.joblib')
# dump(F_sfs, 'M_sfs_pure.joblib')

In [ ]:
%matplotlib inline

from joblib import load
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as plticker
from matplotlib.patches import PathPatch
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

# import file
M_sfs_sample = load("M_sfs_sample.joblib")
# F_sfs_pure = load("F_sfs_pure.joblib")

M_dict_list = list(M_sfs_sample.get_metric_dict().values())
# F_dict_list = list(F_sfs_pure.get_metric_dict().values())

M_score = []
# F_score = []

for index in range(len(M_dict_list)):
    M_score.append(M_dict_list[index]['avg_score'])
#     F_score.append(F_dict_list[index]['avg_score'])

def adjust_box_widths(g, fac):
    """
    Adjust the withs of a seaborn-generated boxplot.
    """

    # iterating through Axes instances
    for ax in g.axes:

        # iterating through axes artists:
        for c in ax.get_children():

            # searching for PathPatches
            if isinstance(c, PathPatch):
                # getting current width of box:
                p = c.get_path()
                verts = p.vertices
                verts_sub = verts[:-1]
                xmin = np.min(verts_sub[:, 0])
                xmax = np.max(verts_sub[:, 0])
                xmid = 0.5*(xmin+xmax)
                xhalf = 0.5*(xmax - xmin)

                # setting new width of box
                xmin_new = xmid-fac*xhalf
                xmax_new = xmid+fac*xhalf
                verts_sub[verts_sub[:, 0] == xmin, 0] = xmin_new
                verts_sub[verts_sub[:, 0] == xmax, 0] = xmax_new

                # setting new width of median line
                for l in ax.lines:
                    if np.all(l.get_xdata() == [xmin, xmax]):
                        l.set_xdata([xmin_new, xmax_new])


y_loc = plticker.MultipleLocator(base=0.1)
fig, (ax1, ax2, ax3, ax4) = plt.subplots(2, 2, figsize=(22, 11))
plt.subplots_adjust(wspace=0)
sns.set(palette='deep')
#sns.set_context("notebook", font_scale=2, rc={"lines.linewidth": 4})
sns.lineplot(x=list(range(1, 31)), y=M_score, palette="Blues", ax = ax1)
# sns.lineplot(x=list(range(1, 31)), y=F_score, palette="Blues", ax = ax2)
ax1.title.set_text('Males')
ax2.title.set_text('Females')
ax1.set_xlabel('Number of variables', fontsize=16)
ax2.set_xlabel('Number of variables', fontsize=16)
ax1.set_ylabel('f1 score')
ax2.set_ylabel('')
ax1.set(ylim=(0, 1))
ax2.set(ylim=(0, 1))
ax1.yaxis.set_major_locator(y_loc)
ax2.yaxis.set_major_locator(y_loc)
ax1.grid(b=True, which='major', axis='y')
ax2.grid(b=True, which='major', axis='y')
plt.setp(ax2.get_yticklabels(), visible=False)
fig.suptitle('Sequential floating forward selection', fontsize=24)

adjust_box_widths(fig, 0.8)
plt.savefig('sffs.png', bbox_inches='tight')

In [ ]:
M_sfs_sample = load("M_sfs_sample.joblib")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
fig1 = plot_sfs(M_sfs_sample.get_metric_dict(),
                kind='std_dev',
                figsize=(22, 11))

plt.ylim([0, 1])
plt.ylabel('f1 score')
plt.title('Sequential Forward Selection')
plt.grid()
plt.show

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

plot_sfs(M_sfs_sample.get_metric_dict(),
                kind='std_dev',
                figsize=(22, 11))

plt.ylim([0, 1])
plt.xlabel('Number of variables', fontsize=20)
plt.ylabel('f1 score', fontsize=20)
plt.title('Sequential Floating Forward Selection - Males', fontsize=20)
plt.grid()
plt.savefig('M_sfs_sample.jpg')



In [ ]:
dict_list = list(M_sfs_sample_pure.get_metric_dict().values())
score = []
for index in range(len(dict_list)):
    score.append(dict_list[index]['avg_score'])

In [ ]:
sns.lineplot(x=list(range(1, 31)), y=M_score, palette="Blues", ax = ax1)